In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Încărcare date [cite: 19]
try:
    df = pd.read_csv('date_promovare_examen.csv')
except FileNotFoundError:
    print("Eroare: Fișierul date_promovare_examen.csv nu a fost găsit.")

# Afișăm primele rânduri pentru verificare
print(df.head())

# Dacă coloanele au nume complicate, le redenumim pentru ușurință (modifică dacă e cazul)
# Presupunem ordinea: Ore Studiu, Ore Somn, Promovare (0/1)
# df.columns = ['Studiu', 'Somn', 'Promovare'] 

# Extragem variabilele [cite: 16, 17, 18]
# Ajustează numele 'Studiu'/'Somn'/'Promovare' dacă în CSV sunt diferite
X_studiu = df['Studiu'].values 
X_somn = df['Somn'].values
y_obs = df['Promovare'].values

# a) Modelul de Regresie Logistică [cite: 20]
with pm.Model() as model_logistic:
    # Priori
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta_studiu = pm.Normal('beta_studiu', mu=0, sigma=2)
    beta_somn = pm.Normal('beta_somn', mu=0, sigma=2)
    
    # Logit și Probabilitate
    mu = alpha + beta_studiu * X_studiu + beta_somn * X_somn
    p = pm.math.sigmoid(mu)
    
    # Likelihood (Bernoulli pentru clasificare binară)
    y = pm.Bernoulli('y', p=p, observed=y_obs)
    
    # Inference
    idata = pm.sample(1000, return_inferencedata=True, progressbar=True)

az.plot_trace(idata, var_names=['alpha', 'beta_studiu', 'beta_somn'])
plt.show()

# b) Granița de decizie (Decision Boundary) [cite: 21]
post = idata.posterior.stack(samples=("chain", "draw"))
alpha_mean = post['alpha'].mean().item()
beta_s_mean = post['beta_studiu'].mean().item()
beta_sleep_mean = post['beta_somn'].mean().item()

print(f"Decision Boundary: {alpha_mean:.2f} + {beta_s_mean:.2f}*Studiu + {beta_sleep_mean:.2f}*Somn = 0")

plt.figure(figsize=(10, 6))
# Scatter plot cu datele
mask_promovat = y_obs == 1
plt.scatter(X_studiu[~mask_promovat], X_somn[~mask_promovat], color='red', label='Picat')
plt.scatter(X_studiu[mask_promovat], X_somn[mask_promovat], color='blue', label='Promovat')

# Linia de decizie
x_vals = np.linspace(X_studiu.min(), X_studiu.max(), 100)
y_vals = (-alpha_mean - beta_s_mean * x_vals) / beta_sleep_mean
plt.plot(x_vals, y_vals, 'k--', label='Decision Boundary')

plt.xlabel('Ore Studiu')
plt.ylabel('Ore Somn')
plt.legend()
plt.show()

# c) Influența [cite: 22]
print(f"Coeficient Studiu: {beta_s_mean:.3f}")
print(f"Coeficient Somn: {beta_sleep_mean:.3f}")
if abs(beta_s_mean) > abs(beta_sleep_mean):
    print("Concluzie: Orele de studiu au o influență mai mare asupra promovării.")
else:
    print("Concluzie: Orele de somn au o influență mai mare asupra promovării.")